In [ ]:
!pip uninstall -y numpy -q
!pip install -q --no-cache-dir numpy==2.0.2

!pip install -q --no-cache-dir \
  scipy==1.11.4 scikit-learn==1.4.2 pandas==2.2.2 \
  joblib==1.4.2 matplotlib==3.8.4 pyyaml==6.0.2 tqdm==4.67.1

import os, sys
os.kill(os.getpid(), 9)


In [ ]:
import numpy, scipy, sklearn, pandas as pd, joblib, matplotlib, yaml
print("NumPy", numpy.__version__+",",
      "SciPy", scipy.__version__+",",
      "sklearn", sklearn.__version__+",",
      "pandas", pd.__version__+",",
      "matplotlib", matplotlib.__version__+",",
      "PyYAML", yaml.__version__)


In [ ]:
from google.colab import drive
drive.mount("/content/drive")


In [ ]:
!ls "/content/drive/MyDrive" | head -n 20


/content/drive/MyDrive/train.csv


In [ ]:
import pandas as pd
import os
CSV_PATH = "/content/drive/MyDrive/train.csv"

assert os.path.exists(CSV_PATH), f"train.csv not found at: {CSV_PATH}"

df_raw = pd.read_csv(CSV_PATH)
print("Dataset loaded:", df_raw.shape)
df_raw.head(10)


In [ ]:
print("Columns:", df_raw.columns.tolist())
print("\n Null values per column:")
print(df_raw.isna().sum())


In [ ]:
df_raw = df_raw.dropna(subset=["comment_text"]).reset_index(drop=True)
print("Cleaned dataset:", df_raw.shape)


In [ ]:
import re

def normalize_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"http\S+", " ", text)           # remove URLs
    text = re.sub(r"[^a-z\s]", " ", text)          # keep only letters/spaces
    text = re.sub(r"\s+", " ", text).strip()       # collapse extra spaces
    return text

df_raw["text_clean"] = df_raw["comment_text"].astype(str).map(normalize_text)

df_raw[["comment_text", "text_clean"]].head(10)


In [ ]:
label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# sanity check
assert all(col in df_raw.columns for col in label_cols), "Missing expected label columns."

# 1 if any toxic label is 1, else 0
df_raw["foul"] = (df_raw[label_cols].sum(axis=1) > 0).astype(int)

print("Binary foul label Ready")
df_raw["foul"].value_counts(normalize=True)


In [ ]:
from sklearn.model_selection import train_test_split

# Features & target
X = df_raw["text_clean"]
y = df_raw["foul"]

# 70 / 15 / 15 split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
X_val, X_test, y_val, y_test   = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp)

print(f"Split complete: train={len(X_train)}, val={len(X_val)}, test={len(X_test)}")


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=30000,       # cap feature size
    ngram_range=(1,2),        # unigrams + bigrams
    min_df=3,                 # ignore rare terms
    max_df=0.90,              # ignore extremely frequent words
    sublinear_tf=True,        # better scaling for long texts
    stop_words="english"
)

X_train_vec = tfidf.fit_transform(X_train)
X_val_vec   = tfidf.transform(X_val)
X_test_vec  = tfidf.transform(X_test)

print("TF-IDF shapes:")
print("Train:", X_train_vec.shape, "Val:", X_val_vec.shape, "Test:", X_test_vec.shape)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

import numpy as np
import pandas as pd


In [ ]:
def evaluate_model(name, model, X_val, y_val, y_pred, y_proba=None):
    acc = accuracy_score(y_val, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_val, y_pred, average="macro", zero_division=0)
    prec_w, rec_w, f1_w, _ = precision_recall_fscore_support(y_val, y_pred, average="weighted", zero_division=0)
    auc = roc_auc_score(y_val, y_proba) if y_proba is not None else np.nan

    print(f"🔹 {name}")
    print(f"   Accuracy: {acc:.4f}")
    print(f"   Macro Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")
    print(f"   Weighted Precision: {prec_w:.4f}, Recall: {rec_w:.4f}, F1: {f1_w:.4f}")
    print(f"   ROC-AUC: {auc:.4f}")
    print("-"*60)

    return {
        "Model": name,
        "Accuracy": acc,
        "Precision_macro": prec,
        "Recall_macro": rec,
        "F1_macro": f1,
        "Precision_weighted": prec_w,
        "Recall_weighted": rec_w,
        "F1_weighted": f1_w,
        "ROC-AUC": auc
    }


In [ ]:
results = []

# 1. Logistic Regression
logreg = LogisticRegression(max_iter=300)
logreg.fit(X_train_vec, y_train)
y_pred = logreg.predict(X_val_vec)
y_proba = logreg.predict_proba(X_val_vec)[:,1]
results.append(evaluate_model("Logistic Regression", logreg, X_val, y_val, y_pred, y_proba))

# 2. Linear SVM (note: no probability output)
svm = LinearSVC()
svm.fit(X_train_vec, y_train)
y_pred = svm.predict(X_val_vec)
results.append(evaluate_model("Linear SVM", svm, X_val, y_val, y_pred))

# 3. Multinomial Naïve Bayes
nb = MultinomialNB()
nb.fit(X_train_vec, y_train)
y_pred = nb.predict(X_val_vec)
y_proba = nb.predict_proba(X_val_vec)[:,1]
results.append(evaluate_model("Multinomial NB", nb, X_val, y_val, y_pred, y_proba))

# 4. Random Forest
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
rf.fit(X_train_vec, y_train)
y_pred = rf.predict(X_val_vec)
y_proba = rf.predict_proba(X_val_vec)[:,1]
results.append(evaluate_model("Random Forest", rf, X_val, y_val, y_pred, y_proba))

# 5. Gradient Boosting (fast & memory-efficient)
gb = HistGradientBoostingClassifier(max_iter=100, random_state=42)
gb.fit(X_train_vec.toarray(), y_train)  # requires dense input
y_pred = gb.predict(X_val_vec.toarray())
y_proba = gb.predict_proba(X_val_vec.toarray())[:,1]
results.append(evaluate_model("Gradient Boosting", gb, X_val, y_val, y_pred, y_proba))

# show summary table
pd.DataFrame(results).set_index("Model").sort_values("F1_macro", ascending=False)


RAM CRASH

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

CSV_PATH = "/content/drive/MyDrive/train.csv"
df_raw = pd.read_csv(CSV_PATH)
label_cols = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
df_raw["text_clean"] = df_raw["comment_text"].astype(str).str.lower()
df_raw["foul"] = (df_raw[label_cols].sum(axis=1) > 0).astype(int)

X = df_raw["text_clean"]
y = df_raw["foul"]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=30000, ngram_range=(1,2), min_df=3, max_df=0.9, sublinear_tf=True, stop_words="english")
X_train_vec = tfidf.fit_transform(X_train)
X_val_vec = tfidf.transform(X_val)

print("TF-IDF matrices:", X_train_vec.shape, X_val_vec.shape)


In [ ]:
# Gradient Boosting on reduced features via TruncatedSVD

import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

# Trying to reuse evaluate_model if it exists; otherwise define a minimal one
try:
    evaluate_model
except NameError:
    def evaluate_model(name, model, X_val, y_val, y_pred, y_proba=None):
        acc = accuracy_score(y_val, y_pred)
        prec, rec, f1, _ = precision_recall_fscore_support(y_val, y_pred, average="macro", zero_division=0)
        prec_w, rec_w, f1_w, _ = precision_recall_fscore_support(y_val, y_pred, average="weighted", zero_division=0)
        auc = roc_auc_score(y_val, y_proba) if y_proba is not None else np.nan
        print(f"🔹 {name}")
        print(f"   Accuracy: {acc:.4f}")
        print(f"   Macro Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")
        print(f"   Weighted Precision: {prec_w:.4f}, Recall: {rec_w:.4f}, F1: {f1_w:.4f}")
        print(f"   ROC-AUC: {auc:.4f}")
        print("-"*60)
        return {
            "Model": name,
            "Accuracy": acc,
            "Precision_macro": prec,
            "Recall_macro": rec,
            "F1_macro": f1,
            "Precision_weighted": prec_w,
            "Recall_weighted": rec_w,
            "F1_weighted": f1_w,
            "ROC-AUC": auc
        }


try:
    results
except NameError:
    results = []


n_components = 512
svd = TruncatedSVD(n_components=n_components, random_state=42)
X_train_red = svd.fit_transform(X_train_vec)
X_val_red   = svd.transform(X_val_vec)

gb = HistGradientBoostingClassifier(max_iter=150, random_state=42)
gb.fit(X_train_red, y_train)
y_pred = gb.predict(X_val_red)
# use [:,1] for positive class prob
y_proba = gb.predict_proba(X_val_red)[:, 1]

results.append(evaluate_model(f"Gradient Boosting (SVD {n_components})", gb, X_val, y_val, y_pred, y_proba))

# Summary table
summary = pd.DataFrame(results).set_index("Model").sort_values("F1_macro", ascending=False)
print("\n Validation Summary (all models):")
display(summary)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import numpy as np, pandas as pd

def evaluate_model(name, model, X_val, y_val, y_pred, y_proba=None):
    acc = accuracy_score(y_val, y_pred)
    p_m, r_m, f1_m, _ = precision_recall_fscore_support(y_val, y_pred, average="macro", zero_division=0)
    p_w, r_w, f1_w, _ = precision_recall_fscore_support(y_val, y_pred, average="weighted", zero_division=0)
    auc = roc_auc_score(y_val, y_proba) if y_proba is not None else np.nan
    print(f" {name}: acc={acc:.4f}, F1_macro={f1_m:.4f}, F1_weighted={f1_w:.4f}, AUC={auc:.4f}")
    return {"Model": name, "Accuracy": acc, "F1_macro": f1_m, "F1_weighted": f1_w, "ROC-AUC": auc}

results = []

# 1. Logistic Regression
logreg = LogisticRegression(max_iter=300, solver="saga", n_jobs=-1, random_state=42)
logreg.fit(X_train_vec, y_train)
results.append(evaluate_model("Logistic Regression",
                              logreg, X_val, y_val,
                              logreg.predict(X_val_vec),
                              logreg.predict_proba(X_val_vec)[:,1]))

# 2. Linear SVM
svm = LinearSVC(random_state=42)
svm.fit(X_train_vec, y_train)
results.append(evaluate_model("Linear SVM", svm, X_val, y_val, svm.predict(X_val_vec)))

# 3. Multinomial NB
nb = MultinomialNB()
nb.fit(X_train_vec, y_train)
results.append(evaluate_model("Multinomial NB",
                              nb, X_val, y_val,
                              nb.predict(X_val_vec),
                              nb.predict_proba(X_val_vec)[:,1]))

# 4. Random Forest (reduced size)
rf = RandomForestClassifier(n_estimators=100, max_depth=12, n_jobs=-1, random_state=42)
rf.fit(X_train_red, y_train)
results.append(evaluate_model("Random Forest (SVD 512)",
                              rf, X_val, y_val,
                              rf.predict(X_val_red),
                              rf.predict_proba(X_val_red)[:,1]))

# 5. Gradient Boosting (SVD 512)
try:
    gb, X_val_red
    y_pred = gb.predict(X_val_red)
    y_proba = gb.predict_proba(X_val_red)[:,1]
    results.append(evaluate_model("Gradient Boosting (SVD 512)", gb, X_val, y_val, y_pred, y_proba))
except Exception as e:


summary = pd.DataFrame(results).set_index("Model").sort_values("F1_macro", ascending=False)
print("\n Validation Summary:")
display(summary)


In [ ]:
import numpy as np, pandas as pd, joblib
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, average_precision_score,
    precision_recall_fscore_support, roc_curve, precision_recall_curve
)
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

logreg = LogisticRegression(max_iter=1000, solver="saga", n_jobs=-1, random_state=42)
logreg.fit(X_train_vec, y_train)
p_val = logreg.predict_proba(X_val_vec)[:, 1]


def choose_threshold(probs, y_true, recall_target=0.80):
    best = {"thr":0.5, "precision":0.0, "recall":0.0, "f1":0.0}
    for thr in np.linspace(0.05, 0.95, 19):
        y_hat = (probs >= thr).astype(int)
        p, r, f1, _ = precision_recall_fscore_support(y_true, y_hat, average="binary", zero_division=0)

        if r >= recall_target and (p > best["precision"] or (p==best["precision"] and f1 > best["f1"])):
            best = {"thr": float(thr), "precision": float(p), "recall": float(r), "f1": float(f1)}

    if best["precision"] == 0.0 and best["recall"] == 0.0:
        y_hat = (probs >= 0.5).astype(int)
        p, r, f1, _ = precision_recall_fscore_support(y_true, y_hat, average="binary", zero_division=0)
        best = {"thr": 0.5, "precision": float(p), "recall": float(r), "f1": float(f1)}
    return best

thr_info = choose_threshold(p_val, y_val, recall_target=0.80)
THR = thr_info["thr"]
print(f"Chosen threshold on validation: {THR:.2f}  (precision={thr_info['precision']:.3f}, recall={thr_info['recall']:.3f}, f1={thr_info['f1']:.3f})")

# 3) Refit TF-IDF + LogReg on TRAIN+VAL, Then TEST
X_trval = pd.concat([X_train, X_val], axis=0)
y_trval = pd.concat([y_train, y_val], axis=0)

tfidf_final = TfidfVectorizer(
    max_features=30000, ngram_range=(1,2), min_df=3, max_df=0.90,
    sublinear_tf=True, stop_words="english"
)
X_trval_vec = tfidf_final.fit_transform(X_trval)
X_test_vec  = tfidf_final.transform(X_test)

logreg_final = LogisticRegression(max_iter=1000, solver="saga", n_jobs=-1, random_state=42)
logreg_final.fit(X_trval_vec, y_trval)

p_test = logreg_final.predict_proba(X_test_vec)[:, 1]
y_hat  = (p_test >= THR).astype(int)

# 4) Full test metrics
report = classification_report(y_test, y_hat, target_names=["proper(0)","foul(1)"], zero_division=0)
cm = confusion_matrix(y_test, y_hat)
roc = roc_auc_score(y_test, p_test)
pr_auc = average_precision_score(y_test, p_test)

p_m, r_m, f1_m, _ = precision_recall_fscore_support(y_test, y_hat, average="macro", zero_division=0)
p_w, r_w, f1_w, _ = precision_recall_fscore_support(y_test, y_hat, average="weighted", zero_division=0)
acc = (y_test == y_hat).mean()

print("\n=== TEST METRICS (Logistic Regression @ tuned threshold) ===")
print(f"Accuracy: {acc:.4f}")
print(f"Macro  P/R/F1: {p_m:.4f} / {r_m:.4f} / {f1_m:.4f}")
print(f"Weight P/R/F1: {p_w:.4f} / {r_w:.4f} / {f1_w:.4f}")
print(f"ROC-AUC: {roc:.4f}   PR-AUC: {pr_auc:.4f}")
print("Confusion matrix:\n", cm)
print("\nDetailed classification report:\n", report)

# Pipeline:
pipe = Pipeline([
    ("tfidf", tfidf_final),
    ("clf", logreg_final)
])

import os
os.makedirs("models", exist_ok=True)
WIN_PATH = "models/best_model_logreg_pipeline.pkl"
joblib.dump({"pipeline": pipe, "threshold": THR}, WIN_PATH)
print("\n Saved pipeline to:", WIN_PATH)


In [ ]:
import numpy as np, pandas as pd, joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, average_precision_score, roc_auc_score
from sklearn.pipeline import Pipeline

label_cols = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

Y = df_raw[label_cols].astype(int)

Y_train = Y.loc[X_train.index]
Y_val   = Y.loc[X_val.index]
Y_test  = Y.loc[X_test.index]

tfidf_multi = TfidfVectorizer(
    max_features=30000, ngram_range=(1,2), min_df=3, max_df=0.90,
    sublinear_tf=True, stop_words="english"
)

X_train_vec_m = tfidf_multi.fit_transform(X_train)
X_val_vec_m   = tfidf_multi.transform(X_val)
X_test_vec_m  = tfidf_multi.transform(X_test)

base_lr = LogisticRegression(max_iter=1000, solver="saga", n_jobs=-1, random_state=42)
multi_clf = OneVsRestClassifier(base_lr)
multi_clf.fit(X_train_vec_m, Y_train)

def choose_threshold_per_label(probs_col, y_true_col, recall_target=0.80):
    best = {"thr":0.5, "precision":0.0, "recall":0.0, "f1":0.0}

    for thr in np.linspace(0.05, 0.95, 19):
        y_hat = (probs_col >= thr).astype(int)
        tp = ((y_hat==1) & (y_true_col==1)).sum()
        fp = ((y_hat==1) & (y_true_col==0)).sum()
        fn = ((y_hat==0) & (y_true_col==1)).sum()
        precision = tp / (tp + fp) if (tp+fp)>0 else 0.0
        recall    = tp / (tp + fn) if (tp+fn)>0 else 0.0
        f1        = (2*precision*recall)/(precision+recall) if (precision+recall)>0 else 0.0
        if recall >= recall_target and (precision > best["precision"] or (precision==best["precision"] and f1>best["f1"])):
            best = {"thr": float(thr), "precision": float(precision), "recall": float(recall), "f1": float(f1)}
    if best["precision"]==0.0 and best["recall"]==0.0:

        thr = 0.5
        y_hat = (probs_col >= thr).astype(int)
        tp = ((y_hat==1) & (y_true_col==1)).sum()
        fp = ((y_hat==1) & (y_true_col==0)).sum()
        fn = ((y_hat==0) & (y_true_col==1)).sum()
        precision = tp / (tp + fp) if (tp+fp)>0 else 0.0
        recall    = tp / (tp + fn) if (tp+fn)>0 else 0.0
        f1        = (2*precision*recall)/(precision+recall) if (precision+recall)>0 else 0.0
        best = {"thr": thr, "precision": precision, "recall": recall, "f1": f1}
    return best


P_val = multi_clf.predict_proba(X_val_vec_m)
per_label_thr = {}
per_label_stats = []
for i, lab in enumerate(label_cols):
    info = choose_threshold_per_label(P_val[:, i], Y_val[lab].values, recall_target=0.80)
    per_label_thr[lab] = float(info["thr"])
    per_label_stats.append({"label": lab, **info})
pd.DataFrame(per_label_stats)
print("Chosen per-label thresholds:\n", per_label_thr)


P_test = multi_clf.predict_proba(X_test_vec_m)
Y_hat_test = np.zeros_like(P_test, dtype=int)
for i, lab in enumerate(label_cols):
    Y_hat_test[:, i] = (P_test[:, i] >= per_label_thr[lab]).astype(int)


from sklearn.metrics import f1_score, precision_score, recall_score
macro_f1 = f1_score(Y_test, Y_hat_test, average="macro", zero_division=0)
weighted_f1 = f1_score(Y_test, Y_hat_test, average="weighted", zero_division=0)
macro_prec = precision_score(Y_test, Y_hat_test, average="macro", zero_division=0)
macro_rec  = recall_score(Y_test, Y_hat_test, average="macro", zero_division=0)


roc_aucs = []
pr_aucs  = []
for i, lab in enumerate(label_cols):
    try:
        roc_aucs.append(roc_auc_score(Y_test[lab], P_test[:, i]))
        pr_aucs.append(average_precision_score(Y_test[lab], P_test[:, i]))
    except ValueError:

        pass

print("\n=== MULTI-LABEL TEST METRICS ===")
print(f"Macro Precision: {macro_prec:.4f}  Macro Recall: {macro_rec:.4f}  Macro F1: {macro_f1:.4f}  Weighted F1: {weighted_f1:.4f}")
if roc_aucs:
    print(f"ROC-AUC (macro over labels): {np.mean(roc_aucs):.4f}")
if pr_aucs:
    print(f"PR-AUC  (macro over labels): {np.mean(pr_aucs):.4f}")


pipe_multi = Pipeline([
    ("tfidf", tfidf_multi),
    ("clf", OneVsRestClassifier(LogisticRegression(max_iter=1000, solver='saga', n_jobs=-1, random_state=42)))
])
pipe_multi.fit(pd.concat([X_train, X_val]), pd.concat([Y_train, Y_val]))

import os
os.makedirs("models", exist_ok=True)
ML_PATH = "models/multi_model_pipeline.pkl"
joblib.dump({
    "pipeline": pipe_multi,       # TF-IDF + OneVsRest(LogReg)
    "labels": label_cols,         # label order
    "thresholds": per_label_thr   # per-label tuned thresholds (floats)
}, ML_PATH)
print("\n Saved multi-label pipeline to:", ML_PATH)


In [ ]:
import re, joblib, numpy as np, pandas as pd
from collections import Counter

BIN_ART = joblib.load("models/best_model_logreg_pipeline.pkl")
BIN_PIPE = BIN_ART["pipeline"]
BIN_THR  = float(BIN_ART["threshold"])

ML_ART  = joblib.load("models/multi_model_pipeline.pkl")
ML_PIPE = ML_ART["pipeline"]            # TF-IDF + OneVsRest(LogReg)
LABELS  = list(ML_ART["labels"])
THR     = {k: float(v) for k, v in ML_ART["thresholds"].items()}

def normalize(text: str) -> str:
    t = text.lower()
    t = re.sub(r"https?://\S+|www\.\S+", " ", t)
    t = re.sub(r"[^a-z\s]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

LEX_BAD = {
    # obscene
    "fuck": 1.5, "fucking": 1.5, "shit": 1.0, "bitch": 1.3, "slut": 1.5,
    # harassment / insults
    "idiot": 0.8, "stupid": 0.7, "moron": 0.7, "dumb": 0.6,
    # violence
    "kill": 2.0, "die": 1.5, "murder": 2.2, "hang": 2.0,
    # sexual harassment terms (non-obscene)
    "creep": 0.7, "pervert": 1.0,
}

# Identity/race group keywords
IDENTITY_GROUPS = {
    "black": "Racist language",
    "blacks": "Racist language",
    "asian": "Racist language",
    "asians": "Racist language",
    "jew": "Antisemitic language",
    "jews": "Antisemitic language",
    "muslim": "Islamophobic language",
    "muslims": "Islamophobic language",
    "gay": "Homophobic language",
    "gays": "Homophobic language",
    "lesbian": "Homophobic language",
    "lesbians": "Homophobic language",
}

# Base tags directly tied to labels
BASE_TAGS = {
    "toxic": "Abusive / toxic language",
    "severe_toxic": "Severe abuse",
    "obscene": "Obscene language",
    "threat": "Violence / threats",
    "insult": "Harassment / insult",
    "identity_hate": "Hate speech / identity-based"
}

def detect_identity_subtags(text_tokens):
    """Return additional identity-specific tags like 'Racist language' when identity terms appear."""
    tags = set()
    for tok in text_tokens:
        if tok in IDENTITY_GROUPS:
            tags.add(IDENTITY_GROUPS[tok])
    return sorted(tags)

# Foulness meter (0 - 10)
def foulness_meter_0_10(text_norm: str, probs: np.ndarray, labels: list[str], thresholds: dict[str, float]) -> int:
    """
    Intuition:
      - Start with max(prob) scaled to 0..6
      - Add lexicon intensity (sum of weights for matched bad words, capped) up to ~2.5
      - Add repetition boost if the same bad word repeats (up to ~1.0)
      - Add density bonus: how many labels exceed their thresholds (up to ~1.0)
      - Clip to [0,10]
    """

    base = 6.0 * float(np.max(probs))


    toks = re.findall(r"\b[a-z]+\b", text_norm)
    weights = [LEX_BAD[w] for w in toks if w in LEX_BAD]
    lex_intensity = min(sum(weights), 2.5)


    cnt = Counter([w for w in toks if w in LEX_BAD])
    rep = max(cnt.values()) if cnt else 1
    rep_boost = min(0.5 * (rep - 1), 1.0)


    num_on = sum(float(probs[i]) >= thresholds[labels[i]] for i in range(len(labels)))
    density = min(0.4 * num_on, 1.0)

    score = base + lex_intensity + rep_boost + density
    return int(round(max(0.0, min(10.0, score))))

# tag generation
def analyze_tweet(text: str):
    """
    Returns a dict:
      - binary: prob + tuned label (winner pipeline)
      - subtypes: per-label prob + label (using per-label thresholds)
      - tags: list of human-friendly tags
      - foulness_meter: 0..10 score
    """
    norm = normalize(text)


    p_bin = float(BIN_PIPE.predict_proba([norm])[0, 1])
    bin_label = int(p_bin >= BIN_THR)


    P = ML_PIPE.predict_proba([norm])[0]
    subtypes = {
        lab: {"prob": float(P[i]), "label": int(float(P[i]) >= THR[lab])}
        for i, lab in enumerate(LABELS)
    }


    active = [BASE_TAGS[lab] for lab in LABELS if subtypes[lab]["label"] == 1]

    toks = re.findall(r"\b[a-z]+\b", norm)
    if subtypes.get("identity_hate", {}).get("label", 0) == 1 or any(tok in IDENTITY_GROUPS for tok in toks):
        active.extend(detect_identity_subtags(toks))

        if "Hate speech / identity-based" not in active:
            active.append("Hate speech / identity-based")


    seen = set(); tags = []
    for t in active:
        if t not in seen:
            seen.add(t); tags.append(t)


    meter = foulness_meter_0_10(norm, np.array(P), LABELS, THR)

    return {
        "binary": {"prob": p_bin, "threshold": BIN_THR, "label": bin_label},
        "subtypes": subtypes,
        "tags": tags,
        "foulness_meter": meter
    }

# test
examples = [
    "Have a wonderful day everyone!",
    "you idiot, fuck off",
    "I hope you die tonight",
    "I hate Black people, they should kill each other",
    "stop being stupid stupid stupid",
    "what a creep, pervert vibes",
]

for s in examples:
    out = analyze_tweet(s)
    print("\nTEXT:", s)
    print(" binary:", out["binary"])
    print(" meter:", out["foulness_meter"])
    print(" tags :", out["tags"])

    fired = {k:v for k,v in out["subtypes"].items() if v["label"]==1}
    print(" subtypes_on:", fired if fired else "{}")


In [ ]:
from google.colab import files
files.download("models/best_model_logreg_pipeline.pkl")
files.download("models/multi_model_pipeline.pkl")
